In [1]:
# e5 分组
import pandas as pd
import numpy as np

In [2]:
# Part1 用apply 执行分组基础操作
# 与聚合不同，传递给 DataFrame子集的 apply 可回调，可访问所有列
df1 = pd.DataFrame({'animal':'cat dog cat fish dog cat cat'.split(),\
                    'size':list('SSMMMLL'),\
                    'weight':[8,10,11,1,20,12,12],\
                    'adult':[False] * 5 + [True] * 2})
print(df1)

  animal size  weight  adult
0    cat    S       8  False
1    dog    S      10  False
2    cat    M      11  False
3   fish    M       1  False
4    dog    M      20  False
5    cat    L      12   True
6    cat    L      12   True


In [3]:
a = df1.groupby('animal').apply(lambda x:x['size'][x['weight'].idxmax()])
print(a)

animal
cat     L
dog     M
fish    M
dtype: object


In [4]:
# Part2 使用 get_group
b = df1.groupby(['animal'])
c = b.get_group('cat')
print(c)

  animal size  weight  adult
0    cat    S       8  False
2    cat    M      11  False
5    cat    L      12   True
6    cat    L      12   True


In [5]:
# Part3 为同一分组的不同内容使用 Apply 函数
def GrowUp(x):
    avg_weight = sum(x[x['size'] == 'S']['weight'] * 1.5)
    avg_weight += sum(x[x['size'] == 'M']['weight'] * 1.25)
    avg_weight += sum(x[x['size'] == 'L']['weight'])
    avg_weight /= len(x)
    return pd.Series(['L',avg_weight,True],index=['size','weight','adult'])
a = b.apply(GrowUp)
print(a)

       size   weight  adult
animal                     
cat       L  12.4375   True
dog       L  20.0000   True
fish      L   1.2500   True


In [6]:
# Part4 Apply函数扩展
import functools
s = pd.Series([i / 100 for i in range(1,11)])
print(s)

0    0.01
1    0.02
2    0.03
3    0.04
4    0.05
5    0.06
6    0.07
7    0.08
8    0.09
9    0.10
dtype: float64


In [7]:
def cum_ret(x,y):
    return x * (1+y)
def red(x):
    return functools.reduce(cum_ret,x,1)
s = s.expanding().apply(red,raw=True)
print(s)

0    1.010000
1    1.030200
2    1.061106
3    1.103550
4    1.158728
5    1.228251
6    1.314229
7    1.419367
8    1.547110
9    1.701821
dtype: float64


In [8]:
# Part5 用分组里的剩余值中的平均值 进行替换
df2 = pd.DataFrame({'A':[1,1,2,2],'B':[1,-1,1,2]})
print(df2)

   A  B
0  1  1
1  1 -1
2  2  1
3  2  2


In [9]:
a = df2.groupby('A')
def replace(x):
    mask = x < 0
    return x.where(mask,x[~mask].mean())
b = a.transform(replace)
print(b)

     B
0  1.0
1 -1.0
2  1.5
3  1.5


In [10]:
# Part6 按聚合数据排序
df3 = pd.DataFrame({'code':['foo','bar','baz']*2,\
                    'data':[0.16,-0.21,0.33,0.45,-0.59,0.62],\
                    'flag':[False,True]*3})
print(df3)

  code  data   flag
0  foo  0.16  False
1  bar -0.21   True
2  baz  0.33  False
3  foo  0.45   True
4  bar -0.59  False
5  baz  0.62   True


In [11]:
a = df3.groupby('code')
b = a[['data']].transform(sum).sort_values(by='data')
c = df3.loc[b.index]
print(c)

  code  data   flag
1  bar -0.21   True
4  bar -0.59  False
0  foo  0.16  False
3  foo  0.45   True
2  baz  0.33  False
5  baz  0.62   True


In [12]:
# Part7 创建多个聚合列
df4 = pd.date_range('2014-10-07',periods=10,freq='2min')
print(df4)

DatetimeIndex(['2014-10-07 00:00:00', '2014-10-07 00:02:00',
               '2014-10-07 00:04:00', '2014-10-07 00:06:00',
               '2014-10-07 00:08:00', '2014-10-07 00:10:00',
               '2014-10-07 00:12:00', '2014-10-07 00:14:00',
               '2014-10-07 00:16:00', '2014-10-07 00:18:00'],
              dtype='datetime64[ns]', freq='2T')


In [13]:
df5 = pd.Series(data=range(10),index=df4)
print(df5)

2014-10-07 00:00:00    0
2014-10-07 00:02:00    1
2014-10-07 00:04:00    2
2014-10-07 00:06:00    3
2014-10-07 00:08:00    4
2014-10-07 00:10:00    5
2014-10-07 00:12:00    6
2014-10-07 00:14:00    7
2014-10-07 00:16:00    8
2014-10-07 00:18:00    9
Freq: 2T, dtype: int64


In [14]:
def MyCust(x):
    if len(x) > 2:
        return x[1] * 1.234
    return pd.NaT
dict1 = {'Mean':np.mean,'Max':np.max,'Custom':MyCust}
d = df5.resample('5min').apply(dict1)
print(d)

Mean    2014-10-07 00:00:00        1
        2014-10-07 00:05:00      3.5
        2014-10-07 00:10:00        6
        2014-10-07 00:15:00      8.5
Max     2014-10-07 00:00:00        2
        2014-10-07 00:05:00        4
        2014-10-07 00:10:00        7
        2014-10-07 00:15:00        9
Custom  2014-10-07 00:00:00    1.234
        2014-10-07 00:05:00      NaT
        2014-10-07 00:10:00    7.404
        2014-10-07 00:15:00      NaT
dtype: object


In [15]:
# Part8 为DataFrame 创建值计数列
df6 = pd.DataFrame({'Color':'Red Red Red Blue'.split(),\
                    'Value':[100,150,50,50]})
print(df6)

  Color  Value
0   Red    100
1   Red    150
2   Red     50
3  Blue     50


In [16]:
df6['js']= df6.groupby('Color').transform(len)
print(df6)

  Color  Value  js
0   Red    100   3
1   Red    150   3
2   Red     50   3
3  Blue     50   1


In [17]:
# Part9 基于索引唯一某列不同分组的值
df7 = pd.DataFrame({'line_race':[10,10,8,10,10,8],\
                    'beyer':[99,102,103,103,88,100]},\
                   index=['Last Gunfighter','Last Gunfighter','Last Gunfighter',\
                          'Paynter','Paynter','Paynter'])
print(df7)

                 line_race  beyer
Last Gunfighter         10     99
Last Gunfighter         10    102
Last Gunfighter          8    103
Paynter                 10    103
Paynter                 10     88
Paynter                  8    100


In [18]:
df7['beyer_shifted'] = df7.groupby(level=0)['beyer'].shift(1)
print(df7)

                 line_race  beyer  beyer_shifted
Last Gunfighter         10     99            NaN
Last Gunfighter         10    102           99.0
Last Gunfighter          8    103          102.0
Paynter                 10    103            NaN
Paynter                 10     88          103.0
Paynter                  8    100           88.0


In [19]:
# Part9 选择每组最大值的行
df8 = pd.DataFrame({'host':['other'] * 2 + ['that'] + ['this'] * 2,\
                    'service':['mail']*1 + ['web'] + ['mail']*2 + ['web'],\
                    'no':[1,2,1,2,1]}).set_index(['host','service'])
print(df8)

               no
host  service    
other mail      1
      web       2
that  mail      1
this  mail      2
      web       1


In [20]:
mask = df8.groupby(level=0).agg('idxmax')
a = df8.loc[mask['no']].reset_index()
print(a)

    host service  no
0  other     web   2
1   that    mail   1
2   this    mail   2


In [21]:
# Part10 分割数据
# 分割 DataFrame
df10 = pd.DataFrame({'Case':list('AAABAABAA'),'Data':np.random.randn(9,)})
print(df10)

  Case      Data
0    A -0.590078
1    A  0.171041
2    A  1.456223
3    B -0.983939
4    A  0.346844
5    A  0.510868
6    B  0.473166
7    A -0.553820
8    A -1.141759


In [22]:
dfs = list(zip(*df10.groupby((1 * (df10['Case'] == 'B')).cumsum().\
                             rolling(window=3,min_periods=1).median())))[-1]
print(dfs[0])

  Case      Data
0    A -0.590078
1    A  0.171041
2    A  1.456223
3    B -0.983939


In [23]:
print(dfs[1])

  Case      Data
4    A  0.346844
5    A  0.510868
6    B  0.473166


In [24]:
print(dfs[2])

  Case      Data
7    A -0.553820
8    A -1.141759


In [25]:
# Part11 透视表
# 部分汇总与小计
df11 = pd.DataFrame({'Province':['ON','QC','BC','AL','AL','MN','ON'],\
                     'City':['Toronto','Montreal','Vancouver','Calgary','Edmonton','Winnipeg','Windsor'],\
                     'Sales': [13, 6, 16, 8, 4, 3, 1]})
print(df11)

  Province       City  Sales
0       ON    Toronto     13
1       QC   Montreal      6
2       BC  Vancouver     16
3       AL    Calgary      8
4       AL   Edmonton      4
5       MN   Winnipeg      3
6       ON    Windsor      1


In [26]:
table1 = pd.pivot_table(df11,values=['Sales'],index=['Province'],\
                        columns=['City'],aggfunc=np.sum,margins=True)
a = table1.stack('City')
print(a)

                    Sales
Province City            
AL       All         12.0
         Calgary      8.0
         Edmonton     4.0
BC       All         16.0
         Vancouver   16.0
MN       All          3.0
         Winnipeg     3.0
ON       All         14.0
         Toronto     13.0
         Windsor      1.0
QC       All          6.0
         Montreal     6.0
All      All         51.0
         Calgary      8.0
         Edmonton     4.0
         Montreal     6.0
         Toronto     13.0
         Vancouver   16.0
         Windsor      1.0
         Winnipeg     3.0


In [27]:
# Part12 按年生成 DataFrame
# 跨列表创建年月
df12 = pd.DataFrame({'value':np.random.randn(36,)},\
                    index=pd.date_range('2011-01-01',freq='M',periods=36))
print(df12)

               value
2011-01-31 -0.745381
2011-02-28  0.425475
2011-03-31  0.882151
2011-04-30 -2.580874
2011-05-31  1.451151
2011-06-30  0.314341
2011-07-31  0.203761
2011-08-31  2.432500
2011-09-30  0.583188
2011-10-31 -1.389719
2011-11-30 -1.768975
2011-12-31  1.326899
2012-01-31  1.223140
2012-02-29 -0.930776
2012-03-31  0.295680
2012-04-30  0.086003
2012-05-31 -0.810416
2012-06-30 -0.223578
2012-07-31  0.391744
2012-08-31 -1.080558
2012-09-30  1.193958
2012-10-31  0.776882
2012-11-30  0.021054
2012-12-31  1.122681
2013-01-31  0.413402
2013-02-28  0.479990
2013-03-31  0.656405
2013-04-30 -0.027017
2013-05-31 -1.050855
2013-06-30 -0.874881
2013-07-31 -1.359947
2013-08-31  0.197538
2013-09-30  0.190300
2013-10-31  0.037138
2013-11-30  0.121957
2013-12-31 -0.209781


In [28]:
a = pd.pivot_table(df12,index=df12.index.month,columns=df12.index.year,\
                   values='value',aggfunc='sum')
print(a)

        2011      2012      2013
1  -0.745381  1.223140  0.413402
2   0.425475 -0.930776  0.479990
3   0.882151  0.295680  0.656405
4  -2.580874  0.086003 -0.027017
5   1.451151 -0.810416 -1.050855
6   0.314341 -0.223578 -0.874881
7   0.203761  0.391744 -1.359947
8   2.432500 -1.080558  0.197538
9   0.583188  1.193958  0.190300
10 -1.389719  0.776882  0.037138
11 -1.768975  0.021054  0.121957
12  1.326899  1.122681 -0.209781


In [29]:
# Part13 Apply函数
# 把嵌入列表 转换为 多层索引 DataFrame
df13 = pd.DataFrame({'A':[[2,4,8,16],[100,200],[10,20,30]],\
                     'B':[['a','b','c'],['jj','kk'],['ccc']]},\
                    index=['I','II','III'])
print(df13)

                 A          B
I    [2, 4, 8, 16]  [a, b, c]
II      [100, 200]   [jj, kk]
III   [10, 20, 30]      [ccc]


In [30]:
def SeriesFromSubList(x):
    return pd.Series(x)
a = pd.concat({ind:row.apply(SeriesFromSubList) for ind,row in df13.iterrows()})
print(a)

         0    1    2     3
I   A    2    4    8  16.0
    B    a    b    c   NaN
II  A  100  200  NaN   NaN
    B   jj   kk  NaN   NaN
III A   10   20   30   NaN
    B  ccc  NaN  NaN   NaN


In [31]:
# 返回 Series
df14 = pd.DataFrame(np.random.randn(2000,2) / 10000,\
                    index=pd.date_range('2001-01-01',periods=2000),\
                    columns=['A',"B"])
print(df14)

                   A         B
2001-01-01  0.000110  0.000088
2001-01-02  0.000088  0.000167
2001-01-03  0.000074 -0.000259
2001-01-04  0.000026 -0.000013
2001-01-05  0.000005 -0.000021
...              ...       ...
2006-06-19 -0.000017 -0.000012
2006-06-20  0.000006 -0.000095
2006-06-21  0.000007  0.000081
2006-06-22  0.000132 -0.000024
2006-06-23 -0.000038  0.000050

[2000 rows x 2 columns]


In [32]:
def gm(df,const):
    v = ((((df.A + df.B) + 1).cumprod())-1) * const
    return v.iloc[-1]
s = pd.Series({df14.index[i]:gm(df14.iloc[i:min(i + 51,len(df14) - 1)],5) for i in range(len(df14) - 50)})
print(s)

2001-01-01    0.014827
2001-01-02    0.014640
2001-01-03    0.013233
2001-01-04    0.013968
2001-01-05    0.013234
                ...   
2006-04-30   -0.009307
2006-05-01   -0.010029
2006-05-02   -0.010435
2006-05-03   -0.011174
2006-05-04   -0.010466
Length: 1950, dtype: float64


In [33]:
# Part14 返回标量值
# Rolling Apply to multiple columns where function returns a Scalar (Volume Weighted Average Price)
# 对多列执行滚动 Apply，函数返回标量值（成交价加权平均价）
rng = pd.date_range(start='2014-01-01', periods=100)
df15 = pd.DataFrame({'Open': np.random.randn(len(rng)),\
                     'Close': np.random.randn(len(rng)),\
                     'Volume': np.random.randint(100, 2000, len(rng))},\
                    index=rng)
print(df15)

                Open     Close  Volume
2014-01-01 -0.057045  0.304154     639
2014-01-02  0.795727  0.791772    1794
2014-01-03  0.564152 -0.191377     678
2014-01-04 -1.406398  0.047722    1800
2014-01-05  0.682064 -0.028551    1335
...              ...       ...     ...
2014-04-06  0.505301  0.018029     449
2014-04-07  0.402810 -1.501956    1889
2014-04-08 -1.001961 -1.482499    1301
2014-04-09  2.358300 -1.417115    1945
2014-04-10  0.248750 -0.197447    1194

[100 rows x 3 columns]


In [34]:
def vwap(bars):
    return ((bars.Close * bars.Volume).sum() / bars.Volume.sum())

window = 5

s = pd.concat([(pd.Series(vwap(df15.iloc[i:i + window]),\
                          index=[df15.index[i + window]])) for i in range(len(df15) - window)])
print(s.round(2))

2014-01-06    0.25
2014-01-07    0.43
2014-01-08    0.52
2014-01-09    0.75
2014-01-10    0.45
              ... 
2014-04-06   -0.11
2014-04-07    0.39
2014-04-08   -0.14
2014-04-09   -0.39
2014-04-10   -0.85
Length: 95, dtype: float64
